In [141]:
import os
import uuid
import numpy as np 
import pandas as pd
#import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
# This is used in order to show the plotted figures within this notebook
%matplotlib inline 
# This is used in order to show the tensorboard within this notebook             
#%load_ext tensorboard  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import Counter
from math import ceil
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_roc_curve
from sklearn.model_selection import learning_curve
from sklearn.multioutput import MultiOutputClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from evaluate import run
import datetime
import numpy as np
import pandas as pd
import seaborn as sns

In [142]:
data = pd.read_csv('output/loan_dev_transformed.csv')
data_comp = pd.read_csv('output/loan_comp_transformed.csv')
data

,Id,loan_date,loan_duration,loan_payments,Predicted,account_frequency,account_district_region,account_district_no_inhabitants,account_district_no_municipalities_0_499,account_district_no_municipalities_500_1999,...,last_trans_balance,mean_trans_balance,mean_trans_amount_absolute,mean_trans_amount_credit,mean_trans_amount_withdrawal,mean_trans_amount_signed,owner_male,owner_age,account_age_months,has_disponent
0,5314,930705,12,1.772411,1.0,1,7,-0.562709,-0.696637,-0.532164,...,-1.105683,-2.341062,-1.095832,-1.277012,-3.548971,2.064417,0,0.738673,-1.637968,0
1,5316,930711,36,0.209593,0.0,2,3,-0.284690,0.251722,-0.103896,...,0.620435,0.662960,0.445111,0.304714,0.259442,0.870102,1,-0.939421,-1.451810,0
2,6863,930728,60,-0.928164,0.0,2,3,-0.912246,1.315032,-0.165077,...,-1.091767,-0.997871,-0.973106,-1.279854,-0.405514,0.362060,1,1.697584,-1.265651,0
3,5325,930803,36,-0.553325,0.0,2,2,-0.352746,1.286294,0.446734,...,-0.183937,-0.150482,-0.193451,-0.489775,-0.135044,0.842490,0,1.377947,-1.079493,0
4,7240,930906,60,0.195440,0.0,1,4,1.737197,-1.127709,-1.327519,...,0.153585,1.047898,1.926310,1.743982,1.716445,0.944229,1,-1.738513,-1.079493,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,961212,48,-0.414986,0.0,2,1,0.600503,-0.208089,1.731539,...,0.927616,0.068220,-1.137214,-1.012047,-1.070137,-0.537260,0,-0.859512,1.899044,1
324,5625,961215,60,-0.204510,1.0,2,7,-2.078174,0.366674,-0.715708,...,0.889229,0.900256,0.595847,0.285059,0.536220,0.540540,1,1.697584,0.037459,0
325,6805,961221,48,-1.466909,0.0,2,1,0.828068,-1.127709,-1.205157,...,0.023906,-0.097886,-0.313022,-0.621863,-0.182382,0.514984,0,0.259218,-0.893334,0
326,7233,961225,36,-0.426400,0.0,2,6,-0.578490,0.998913,-0.042715,...,0.188744,1.007047,0.910852,0.869015,0.695252,-0.583636,1,0.578855,1.340569,0


# Outlier Detection

- Standard Deviation
- Z-Score
- Interquartile Range (IQR)
- Percentile

In [143]:
for col in data.columns:
    print("-"*50)
    print(f"Statistics of {col}")
    print("-"*50)
    #data[col].plot.box();

    q3, q1 = np.percentile(data[col], [75 ,25])

    print("Min: ", end="");                 print(data[col].min())
    print("Max: ", end="");                 print(data[col].max())
    print("Mean: ", end="");                print(data[col].mean())
    print("Median: ", end="");              print(data[col].median())
    print("Standard deviation: ", end="");  print(data[col].std())
    print("Skewness: ", end="");            print(data[col].skew())  
    print("Variance: ", end="");            print(data[col].var())
    print("Kurtosis: ", end="");            print(data[col].kurtosis())
    print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

# var() method returns the unbiased variance
# Medium-High std which indicates that the data is spread out and not clustered around the mean
# Skewness > 1 => it is skewed to the right



--------------------------------------------------
Statistics of Id
--------------------------------------------------
Min: 4959
Max: 7308
Mean: 6205.658536585366
Median: 6227.5
Standard deviation: 667.985674726219
Skewness: -0.126359735421936
Variance: 446204.86163944204
Kurtosis: -1.168537695082133
Interquartile Range: 1133.0 | Q1: 5604.25 | Q3: 6737.25
--------------------------------------------------
Statistics of loan_date
--------------------------------------------------
Min: 930705
Max: 961227
Mean: 949989.125
Median: 950565.5
Standard deviation: 9495.504646353156
Skewness: -0.2574182118951275
Variance: 90164608.48891437
Kurtosis: -1.1466194893417647
Interquartile Range: 19716.0 | Q1: 940809.25 | Q3: 960525.25
--------------------------------------------------
Statistics of loan_duration
--------------------------------------------------
Min: 12
Max: 60
Mean: 35.853658536585364
Median: 36.0
Standard deviation: 16.73475166172723
Skewness: 0.05589223008522359
Variance: 280.05191

## Standard Deviation

In [144]:
outliers_std = {}
for col in data.columns:
    data_mean, data_std = data[col].mean(), data[col].std()
    # identify outliers
    cut_off = data_std * 4
    lower, upper = data_mean - cut_off, data_mean + cut_off 
    outliers_std[col] = pd.concat([data.loc[data[col] < lower] , (data.loc[data[col] > upper])])

for key in outliers_std.keys():
    if (len(outliers_std[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_std[key][key])

--------------------------------------------------
Outliers of owner_card_type - mean: 0.0701219512195122 - std: 0.3886272992765306
--------------------------------------------------
33     2
88     2
201    2
244    2
252    2
263    2
271    2
276    2
321    3
322    3
Name: owner_card_type, dtype: int64
--------------------------------------------------
Outliers of count_trans_withdrawal_card - mean: 0.0701219512195122 - std: 0.5888546783181915
--------------------------------------------------
33     5
252    8
321    4
Name: count_trans_withdrawal_card, dtype: int64
--------------------------------------------------
Outliers of count_trans_ksymbol_sanction_interest_if_negative_balance - mean: 0.042682926829268296 - std: 0.20245012240637317
--------------------------------------------------
55     1
62     1
95     1
111    1
123    1
135    1
140    1
144    1
158    1
217    1
223    1
232    1
251    1
277    1
Name: count_trans_ksymbol_sanction_interest_if_negative_balance, dt

In [145]:
outliers_ids_std = set([val for key in outliers_std.keys() for val in outliers_std[key]['Id']])
outlier_ratio = len(outliers_ids_std) / len(data)
outlier_ratio

0.07317073170731707

## Z-Score

In [146]:
outliers_z = {}
for col in data.columns:
    z_val = (data[col] - data[col].mean()) / data[col].std()
    outliers_z[col] = data.loc[(z_val < -4) | (z_val > 4)]

for key in outliers_z.keys():
    if (len(outliers_z[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_z[key][key])

--------------------------------------------------
Outliers of owner_card_type - mean: 0.0701219512195122 - std: 0.3886272992765306
--------------------------------------------------
33     2
88     2
201    2
244    2
252    2
263    2
271    2
276    2
321    3
322    3
Name: owner_card_type, dtype: int64
--------------------------------------------------
Outliers of count_trans_withdrawal_card - mean: 0.0701219512195122 - std: 0.5888546783181915
--------------------------------------------------
33     5
252    8
321    4
Name: count_trans_withdrawal_card, dtype: int64
--------------------------------------------------
Outliers of count_trans_ksymbol_sanction_interest_if_negative_balance - mean: 0.042682926829268296 - std: 0.20245012240637317
--------------------------------------------------
55     1
62     1
95     1
111    1
123    1
135    1
140    1
144    1
158    1
217    1
223    1
232    1
251    1
277    1
Name: count_trans_ksymbol_sanction_interest_if_negative_balance, dt

In [147]:
outliers_ids_z = set([val for key in outliers_z.keys() for val in outliers_z[key]['Id']])
outlier_ratio = len(outliers_ids_z) / len(data)
outlier_ratio

0.07317073170731707

# Dataset without Outliers

In [148]:
outliers_ids = outliers_ids_std.union(outliers_ids_z)
manual_outlier_df = data.loc[~data['Id'].isin(outliers_ids)]
manual_outlier_df

,Id,loan_date,loan_duration,loan_payments,Predicted,account_frequency,account_district_region,account_district_no_inhabitants,account_district_no_municipalities_0_499,account_district_no_municipalities_500_1999,...,last_trans_balance,mean_trans_balance,mean_trans_amount_absolute,mean_trans_amount_credit,mean_trans_amount_withdrawal,mean_trans_amount_signed,owner_male,owner_age,account_age_months,has_disponent
0,5314,930705,12,1.772411,1.0,1,7,-0.562709,-0.696637,-0.532164,...,-1.105683,-2.341062,-1.095832,-1.277012,-3.548971,2.064417,0,0.738673,-1.637968,0
1,5316,930711,36,0.209593,0.0,2,3,-0.284690,0.251722,-0.103896,...,0.620435,0.662960,0.445111,0.304714,0.259442,0.870102,1,-0.939421,-1.451810,0
2,6863,930728,60,-0.928164,0.0,2,3,-0.912246,1.315032,-0.165077,...,-1.091767,-0.997871,-0.973106,-1.279854,-0.405514,0.362060,1,1.697584,-1.265651,0
3,5325,930803,36,-0.553325,0.0,2,2,-0.352746,1.286294,0.446734,...,-0.183937,-0.150482,-0.193451,-0.489775,-0.135044,0.842490,0,1.377947,-1.079493,0
4,7240,930906,60,0.195440,0.0,1,4,1.737197,-1.127709,-1.327519,...,0.153585,1.047898,1.926310,1.743982,1.716445,0.944229,1,-1.738513,-1.079493,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,961212,48,-0.414986,0.0,2,1,0.600503,-0.208089,1.731539,...,0.927616,0.068220,-1.137214,-1.012047,-1.070137,-0.537260,0,-0.859512,1.899044,1
324,5625,961215,60,-0.204510,1.0,2,7,-2.078174,0.366674,-0.715708,...,0.889229,0.900256,0.595847,0.285059,0.536220,0.540540,1,1.697584,0.037459,0
325,6805,961221,48,-1.466909,0.0,2,1,0.828068,-1.127709,-1.205157,...,0.023906,-0.097886,-0.313022,-0.621863,-0.182382,0.514984,0,0.259218,-0.893334,0
326,7233,961225,36,-0.426400,0.0,2,6,-0.578490,0.998913,-0.042715,...,0.188744,1.007047,0.910852,0.869015,0.695252,-0.583636,1,0.578855,1.340569,0


In [ ]:
manual_outlier_df.to_csv('output/loan_dev_transformed_outliers.csv', index=False)